In [ ]:
! pip install torch
! pip install torchvision 
! pip install matplotlib 
! pip install sklearn
! pip install openai-clip 
! pip install nmslib 
! pip install gradio 
! pip install pandas 
! pip install pillow


In [ ]:
import pandas as pd 
image_df = pd.read_parquet("/home/divar/Documents/divar-projects/clip image task/data/images.parquet")

In [ ]:
image_df.head()

# search using clip model

In [ ]:
! pip install nmslib

In [ ]:
from transformers import CLIPProcessor, CLIPModel

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

In [ ]:
import nmslib
import numpy as np

data_embeddings = np.array(image_df['embedding'].tolist())
index = nmslib.init(method='hnsw', space='cosinesimil')
index.addDataPointBatch(data_embeddings)
index.createIndex({'post': 2}, print_progress=True)


In [ ]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def embed_image(image: Image.Image):
    image = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        embedding = model.encode_image(image).cpu().numpy()[0]
    return embedding

def search_image(query_image: Image.Image):
    query_embedding = embed_image(query_image)
    idx, distance = index.knnQuery(query_embedding, k=1)
    return idx[0], distance[0]


In [1]:
# def get_image_embedding(image):
#     inputs = processor(images=image, return_tensors="pt")
#     outputs = model.get_image_features(**inputs)
#     normalized_embeddings = outputs / outputs.norm(p=2, dim=-1, keepdim=True)
#     return normalized_embeddings